Setting Directory

In [1]:
import os
from pathlib import Path
import logging

In [2]:
os.chdir("../")
%pwd

'/Volumes/T7/DL_Skin_Cancer_Project/skin_cancer_diagnosis'

Training Component

Import Confugarations

In [4]:
from skinCancerDiagnosis.entity.config_entity import TrailTrainingConfig
from skinCancerDiagnosis.config.configuration import ConfugarationManager
from skinCancerDiagnosis.components.data_prep import DataGenerator

Necesary LIbraries

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
class Training:
    def __init__(self,base_model,model_name,config:TrailTrainingConfig):
        self.config = config
        self.early_stopping = EarlyStopping(monitor='val_loss',patience=config.params_patience,restore_best_weights=True)
        self.base_model = base_model
        self.model_name = model_name

    def define_base_model(self):
#         base_model = VGG19(weights='imagenet', include_top=False)
        base_model = self.base_model
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(8, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)

        for layer in base_model.layers:
            layer.trainable = False

        print(model.summary())
        
        return model
    
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model,model_name):
        model_path = os.path.join(path,model_name)
        model.save(model_path)
    
    def get_model_and_compile(self):
        with tf.device('/GPU:0'):
            self.model = self.define_base_model()
            self.model.compile(
                optimizer = 'adam',
                loss = "categorical_crossentropy",
                metrics = ['accuracy']
            )
    
    def train_model(self,train_generator,val_generator):
        with tf.device('/GPU:0'):
            self.history = self.model.fit(
                train_generator,
                epochs = self.config.params_epochs,
                validation_data = val_generator,
                validation_steps = val_generator.samples//val_generator.batch_size,
                verbose = 1,
                callbacks = [self.early_stopping]
            )

        self.save_model(
            path=self.config.root_dir,
            model=self.model,
            model_name = self.model_name
        )

VGG19 training Pipeline

In [7]:
from tensorflow.keras.applications import VGG19

In [8]:
try:
    config = ConfugarationManager()

    data_prep_config = config.get_data_prep_config()
    generator = DataGenerator(config=data_prep_config)
    train_generator = generator.get_train_generator()
    val_generator = generator.get_val_generator()
    
    trail_train_config = config.get_trail_training_config()
    vgg_base = VGG19(weights='imagenet', include_top=False)
    vgg_trainer = Training(config=trail_train_config,base_model=vgg_base,model_name='VGG19.h5')
    vgg_trainer.get_model_and_compile()
    vgg_trainer.train_model(train_generator=train_generator,val_generator=val_generator)

except Exception as e:
    raise e

[2024-05-01 11:38:32,896: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-01 11:38:32,900: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-01 11:38:32,901: INFO: common: created directory at: artifacts]


AttributeError: module 'albumentations' has no attribute 'compose'